## Postprocessing randomized measurement on IBM quantum computers (

This tutorial illustrates how to acquire and save randomized measurements on a quantum computer using Qiskit (RandomizedMeasurementsQiskit.ipynb). This data can be then postprocessed using our julia library RandomMeas (RandomizedMeasurementsQiskitPostprocessing.ipynb)

In [ ]:
using RandomMeas
using NPZ

### Loading the data and retrieving the parameters of the experiment

In [25]:
job_id = "fd8feff5-c084-4087-bb38-e580aa9584b4"
local_unitary_ = npzread("data/"*job_id*"_u.npy")
data_cal_ = npzread("data/"*job_id*"_data.npy")[1,:,:,:]; #Data obtained from the circuit qc_meas used for calibrating robust shadows
#data_cal = 3 .*data_cal .- 1;
data_ = npzread("data/"*job_id*"_data.npy")[2,:,:,:]; #Data obtained from the preparation of the cluster state
NU,NM,N = size(data_)
@show NU,NM,N;

(NU, NM, N) = (2048, 1024, 4)


### Converting the data into RandMeas.jl format 

In [34]:
ξ = siteinds("Qubit",N)
data_cal = Vector{MeasurementData{LocalUnitaryMeasurementSetting}}(undef,NU)
data = Vector{MeasurementData{LocalUnitaryMeasurementSetting}}(undef,NU)

for r in 1:NU
    local_unitary = Vector{ITensor}()
    for i in 1:N
        push!(local_unitary,ITensor(local_unitary_[r,i,:,:],ξ[i]',ξ[i]))
    end
    measurement_setting = LocalUnitaryMeasurementSetting(N,local_unitary,ξ)
    data_cal[r] = MeasurementData(data_cal_[r,:,:];measurement_setting=measurement_setting)
    data[r] = MeasurementData(data_[r,:,:];measurement_setting=measurement_setting)
end
group_cal = MeasurementGroup(data_cal)
group = MeasurementGroup(data);

### Extraction of the calibration parameters $G$ for building robust shadows

For purely readout error models, we have $G\approx 1-p$ where $p$ is the readout error 

In [35]:
G_e = zeros(Float64, N)
states = ["Dn" for n in 1:N]
ψ0  = MPS(ComplexF64,ξ,states);
# ------------------------------------------------------------------------------
# Generating Measurement Settings and Data

@showprogress dt=1 for i in 1:N
    reduced_group_cal = reduce_to_subsystem(group_cal, [i])
    
    for r in 1:NU
            # Compute the measured Born probabilities for qubit i using the actual measurement data
        data = reduced_group_cal.measurements[r]
        P_measured = MeasurementProbability(data).measurement_probability
            # Compute the expected Born probabilities for qubit i from the ideal (noise-free) state ψ0.
        P_expected = MeasurementProbability(reduce_to_subsystem(ψ0,collect(i:i)), data.measurement_setting).measurement_probability
        cross_corr = (P_measured * P_expected)[]
        self_corr = (P_expected * P_expected)[]
    
        # Estimate the G value for qubit i using the formula:
        #     G_e[i] = 3 * (mean(cross_corr_first - self_corr_first)) + 1,
        G_e[i] += 3 * (cross_corr-self_corr)/NU + 1/NU
    end
    
end
print("Calibrated RM parameters: ", G_e)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:26


Calibrated RM parameters: [0.9622838739492673, 0.9949359542163281, 0.9606441674183136, 0.9718002683460556]

### Extracting the purity

We extract the purity of the first two qubits, and the purity of the four qubits. As expected for a cluster state, we have $p(12)\approx 0.5$, and $p(1234)\approx 1$.  

In [36]:
ρ = get_dense_shadows(group,number_of_ru_batches=10)
ρ_sub = partial_trace(ρ,collect(1:2))
ρ_mitigated = get_dense_shadows(group,number_of_ru_batches=10;G=G_e)
ρ_mitigated_sub = partial_trace(ρ_mitigated,collect(1:2))
println("SubSystem Purity (unmitigated) ", get_trace_moment(ρ_sub,2))
println("SubSystem Purity (mitigated) ", get_trace_moment(ρ_mitigated_sub,2))
println("Total Purity (unmitigatd) ", get_trace_moment(ρ,2))
println("Total Purity (mitigated) ", get_trace_moment(ρ_mitigated,2))

SubSystem Purity (unmitigated) 0.4615334746665471 + 0.0im
SubSystem Purity (mitigated) 0.5025476630435592 + 0.0im
Total Purity (unmitigatd) 0.7192320271436738 + 0.0im
Total Purity (mitigated) 1.0070165292056668 + 0.0im
